<a href="https://colab.research.google.com/github/ralsouza/apache_spark_real_time_analytics/blob/master/notebooks/11_pyspark_mllib_clustering_k_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering K-Means

**Description:**
*   Unsupervised algorithm;
*   Grouping data by simility;
*   Partitions the data by a `k` number of clusters, with each observation belongs to a just one cluster;
*   The clusterizations is made measuring the distance between the data points and grouping them;
*   Multiple distance measures may be used, such as, `Euclidian Distance` and `Manhattan Distance`;

**Advantages:**
*   Faster;
*   Efficient to many variables;

**Disvantages:**
*   The `k` value must be know;
*   The initial value of `k` influences on created clusters;

**Application:**
*   Priliminary grouping before applying classification techniques;
*   Geographic clustering;



# Setup Spark


In [ ]:
!apt-get update

In [1]:


# Install the dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
# Environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [3]:
# Make pyspark "importable"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

In [4]:
# Libraries and Context Setup
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)


# Instance Spark Session
spark = SparkSession.builder.master('local').appName('spark_ml_lib').getOrCreate()

# Create the SQL Context
sqlContext = pyspark.SQLContext(sc)

# Grouping cars

In [6]:
# Libraries
import math
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors
from pyspark.ml.clustering import KMeans
import matplotlib.pylab as plt
%matplotlib inline

In [7]:
# Spark Session
sp_session = SparkSession.builder.master('local').appName('app_spark_mllib').getOrCreate()

In [8]:
# Load data
rdd_cars = sc.textFile('/content/drive/My Drive/Colab Notebooks/08-apache-spark/data/mllib/carros2.csv')

In [9]:
rdd_cars.cache()

/content/drive/My Drive/Colab Notebooks/08-apache-spark/data/mllib/carros2.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [12]:
# Remove the header
first_row = rdd_cars.first()
rdd_cars2 = rdd_cars.filter(lambda x: x != first_row)
rdd_cars2.count()

197

In [15]:
rdd_cars2.take(5)

['subaru,gas,std,two,hatchback,fwd,four,69,4900,31,36,5118',
 'chevrolet,gas,std,two,hatchback,fwd,three,48,5100,47,53,5151',
 'mazda,gas,std,two,hatchback,fwd,four,68,5000,30,31,5195',
 'toyota,gas,std,two,hatchback,fwd,four,62,4800,35,39,5348',
 'mitsubishi,gas,std,two,hatchback,fwd,four,68,5500,37,41,5389']

In [16]:
# Converting and cleaning
def transform_to_numeric(input_str):
  
  att_list = input_str.split(',')

  doors = 1.0 if att_list[3] == 'two' else 2.0
  body = 1.0 if att_list[4] == 'sedan' else 2.0

  rows = Row(DOORS = doors, BODY = float(body), HP = float(att_list[7]), 
             RPM = float(att_list[8]), MPG = float(att_list[9]))
  
  return rows

In [19]:
# Apply function
rdd_cars3 = rdd_cars2.map(transform_to_numeric)

# Persist in memory, could be .cache()
rdd_cars3.persist()

rdd_cars3.take(3)

[Row(BODY=2.0, DOORS=1.0, HP=69.0, MPG=31.0, RPM=4900.0),
 Row(BODY=2.0, DOORS=1.0, HP=48.0, MPG=47.0, RPM=5100.0),
 Row(BODY=2.0, DOORS=1.0, HP=68.0, MPG=30.0, RPM=5000.0)]

In [27]:
# Transform to dataframe
df_cars = sp_session.createDataFrame(rdd_cars3)
df_cars.show(5)

+----+-----+----+----+------+
|BODY|DOORS|  HP| MPG|   RPM|
+----+-----+----+----+------+
| 2.0|  1.0|69.0|31.0|4900.0|
| 2.0|  1.0|48.0|47.0|5100.0|
| 2.0|  1.0|68.0|30.0|5000.0|
| 2.0|  1.0|62.0|35.0|4800.0|
| 2.0|  1.0|68.0|37.0|5500.0|
+----+-----+----+----+------+
only showing top 5 rows



In [28]:
# Summarizing with Pandas
desc_cars = df_cars.describe().toPandas()

In [29]:
# Show dataframe
desc_cars

,summary,BODY,DOORS,HP,MPG,RPM
0,count,197,197,197,197,197
1,mean,1.532994923857868,1.5685279187817258,103.60406091370558,25.15228426395939,5118.0203045685275
2,stddev,0.5001812579359883,0.49654352778167493,37.639205349518356,6.437862917085915,481.03591405011446
3,min,1.0,1.0,48.0,13.0,4150.0
4,max,2.0,2.0,262.0,49.0,6600.0


In [32]:
# Get means and deviations converting to lists
cars_means = desc_cars.iloc[1,1:5].values.tolist()
cars_devs  =  desc_cars.iloc[2,1:5].values.tolist()

In [33]:
cars_means

['1.532994923857868',
 '1.5685279187817258',
 '103.60406091370558',
 '25.15228426395939']

In [34]:
cars_devs

['0.5001812579359883',
 '0.49654352778167493',
 '37.639205349518356',
 '6.437862917085915']

In [35]:
# Store the variables to the broadcast variables - broadcast variables are read-only
bc_means = sc.broadcast(cars_means)
bc_devs = sc.broadcast(cars_devs)